In [1]:
# the following Dhondt algorithm based on idea from https://gist.github.com/brunosan/96288a8612894fca718aacbcc501ee09
###################  2D inputs                 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
################### have multiple not 1 output !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
import tensorflow as tf
import  tensorflow.keras.backend as K
import  tensorflow.keras as keras
from tensorflow.keras.layers import Input, Concatenate
#
from tensorflow.keras.models import Model
#
#tf.compat.v1.enable_eager_execution()
#@tf.keras.utils.register_keras_serializable()
#
class DHondt_TensorFlow_KERAS_2D(keras.layers.Layer):                        
    def __init__(self,  model_column_number, **kwargs):
        super(DHondt_TensorFlow_KERAS_2D, self).__init__(**kwargs)           
        self.model_column_number = model_column_number
        
    # about get_config  see the following link:
    # https://towardsdatascience.com/how-to-write-a-custom-keras-model-so-that-it-can-be-deployed-for-serving-7d81ace4a1f8
    def get_config(self):
        config = super().get_config()
        # save constructor args
        config['model_column_number'] = self.model_column_number
        # 
        return config
    #
    def build(self, inputShape):
        super(DHondt_TensorFlow_KERAS_2D, self).build(inputShape)

    def compute_output_shape(self,inputShape):
        return [(None,1) for i in self.model_column_number]
    #
    @tf.function
    def call(self, inputs):
        #
        inputs = tf.reshape(inputs, [-1])
        inputs = tf.squeeze(inputs) 
        #
        ##########################################################################
        #
        nSeats = tf.math.reduce_sum(inputs)
        #
        seats = tf.cast(tf.math.floor(inputs), dtype = tf.float32)
        #
        if tf.greater(nSeats, 0):
           votes = tf.divide(inputs, nSeats) 
           #
           nSeats = tf.cast(nSeats, dtype = tf.int32)
           #
           allocated_seat_number = tf.cast(tf.math.reduce_sum(seats), dtype = tf.int32)
           #
           ##########################################################################
           #
           t_votes = tf.identity(votes)
           #
           while tf.math.less(allocated_seat_number, nSeats):
                    #
                    next_seat = tf.math.argmax(t_votes)
                    #
                    ############################################################
                    #
                    update = tf.gather(seats,  next_seat)
                    #
                    update = tf.add(update, 1.0)
                    #
                    update = tf.convert_to_tensor([update])
                    #
                    seats = tf.tensor_scatter_nd_update(seats, [[next_seat]], update)
                    #
                    allocated_seat_number = tf.add(allocated_seat_number, 1)
                    #
                    ############################################################
                    #
                    denumerator = tf.add(update, 1.0)
                    #
                    numerator   = tf.gather(votes,  next_seat)
                    #
                    update      = tf.divide(numerator, denumerator)
                    #
                    t_votes     = tf.tensor_scatter_nd_update(t_votes,  [[next_seat]], update)
           ##########################################################################
           seats = tf.reshape(seats, [-1, self.model_column_number])
        #
        else:
           seats = tf.reshape(tf.zeros(tf.size(inputs)), [-1, self.model_column_number])
        #
        return   [seats[:, i: ( i + 1)] for i in range(self.model_column_number)]  
#
#
#
#
if 0: 
   model_column_number = 2500
   input_layer  = Input(shape = (model_column_number,))
   #
   output_layer =  DHondt_TensorFlow_KERAS_2D(model_column_number)(input_layer) 
   #
   output_layer = Concatenate()(output_layer)
   #
   model = Model(inputs = input_layer, outputs = output_layer)
   #
   model.save("DHondt_TensorFlow_KERAS_2D_" + str(model_column_number) ) 
#
else:
   model_column_number = 4
   #
   input_layer  = Input(shape = (model_column_number,))
   #
   output_layer =  DHondt_TensorFlow_KERAS_2D(model_column_number)(input_layer) 
   #
   output_layer = Concatenate()(output_layer)
   #
   model = Model(inputs = input_layer, outputs = output_layer)
   #
   total_impressions  = 10
   #
   #inputs = tf.constant([[0.025, 0.025, 0.025, 0.025], [0.05, 0.05, 0.05, 0.05], [0.05, 0.05, 0.05, 0.05], [0.125, 0.125, 0.125, 0.125]])
   inputs = tf.constant([[0.03, 0.07, 0.04, 0.06], [0.5, 0.03, 0.07, 0.04], [0.06, 0.05, 0.05, 0.0]])
   inputs = tf.math.multiply(inputs, total_impressions)
   #
   seats = model.predict(inputs)
   # 
   print("Number of allocated seats: ", tf.reduce_sum(seats))
   print("prediction, seats:")
   print(seats)
   #
   print("converted allocation output to vector: ", tf.reshape(seats, [-1, 1]))

1/1 [==============================] - 0s 155ms/step
Number of allocated seats:  tf.Tensor(10.0, shape=(), dtype=float32)
prediction, seats:
[[0. 1. 0. 0.]
 [8. 0. 1. 0.]
 [0. 0. 0. 0.]]
converted allocation output to vector:  tf.Tensor(
[[0.]
 [1.]
 [0.]
 [0.]
 [8.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(12, 1), dtype=float32)
